In [2]:
import os
import numpy as np
import parselmouth
from parselmouth.praat import call
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [6]:
# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치와 강세 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    
    intensity = snd.to_intensity()
    intensity_values = intensity.values.T[0]
    
    return pitch_values, intensity_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity = extract_features(tts_audio_path)
    user_pitch, user_intensity = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

In [7]:
# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)

ValueError: Input vector should be 1-D.

In [8]:
import os
import numpy as np
import parselmouth
from parselmouth.praat import call
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치와 강세 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    
    # 0 Hz 값(무음)을 제거하고 1차원으로 변환
    pitch_values = pitch_values[pitch_values > 0]

    intensity = snd.to_intensity()
    intensity_values = intensity.values.T[0]  # 1차원 배열로 변환
    
    return pitch_values, intensity_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity = extract_features(tts_audio_path)
    user_pitch, user_intensity = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [9]:
import os
import numpy as np
import parselmouth
from parselmouth.praat import call
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치와 강세 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    
    # 0 Hz (무음) 값 제거 및 1차원 배열로 변환
    pitch_values = pitch_values[pitch_values > 0].flatten()

    intensity = snd.to_intensity()
    intensity_values = intensity.values.T.flatten()  # 1차원 배열로 변환
    
    return pitch_values, intensity_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity = extract_features(tts_audio_path)
    user_pitch, user_intensity = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


gTTSError: Failed to connect. Probable cause: Unknown

In [11]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0].flatten()  # 무음 제거 및 1차원 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = intensity.values.T.flatten()  # 1차원 배열로 변환
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.array(formant_values)
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [12]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0].flatten()  # 무음 제거 및 1차원 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = intensity.values.flatten()  # 1차원 배열로 변환
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.array(formant_values).flatten()  # 1차원 배열로 변환
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # feature1과 feature2가 1차원 배열인지 확인 후 변환
    feature1 = feature1.flatten()
    feature2 = feature2.flatten()
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [14]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean


In [18]:
# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0].flatten()  # 무음 제거 및 1차원 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.array(intensity.values.flatten())  # 1차원 배열로 강제 변환
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.array(formant_values).flatten()  # 1차원 배열로 강제 변환
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # feature1과 feature2가 1차원 배열인지 확인 후 변환
    feature1 = np.array(feature1).flatten()
    feature2 = np.array(feature2).flatten()
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

In [19]:
# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)

ValueError: Input vector should be 1-D.

In [17]:
# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0].flatten()  # 무음 제거 및 1차원 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.array(intensity.values.flatten())  # 1차원 배열로 강제 변환
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.array(formant_values).flatten()  # 1차원 배열로 강제 변환
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # feature1과 feature2가 1차원 배열인지 확인 후 변환
    feature1 = np.array(feature1).flatten()
    feature2 = np.array(feature2).flatten()
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)

ValueError: Input vector should be 1-D.

In [20]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]  # 무음 제거
    pitch_values = np.ravel(pitch_values)  # 1차원 배열로 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.ravel(intensity.values)  # 1차원 배열로 변환
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.ravel(np.array(formant_values))  # 1차원 배열로 변환
    
    # 차원 확인
    print(f"Pitch shape: {pitch_values.shape}")
    print(f"Intensity shape: {intensity_values.shape}")
    print(f"Formant shape: {formant_values.shape}")
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # feature1과 feature2가 1차원 배열인지 확인 후 변환
    feature1 = np.ravel(np.array(feature1))
    feature2 = np.ravel(np.array(feature2))
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


Pitch shape: (247,)
Intensity shape: (505,)
Formant shape: (411,)
Pitch shape: (201,)
Intensity shape: (819,)
Formant shape: (662,)


ValueError: Input vector should be 1-D.

In [21]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]  # 무음 제거
    pitch_values = np.ravel(pitch_values)  # 1차원 배열로 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.ravel(intensity.values)  # 1차원 배열로 변환
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.ravel(np.array(formant_values))  # 1차원 배열로 변환
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # 두 벡터의 길이 조정 (짧은 벡터에 맞추기)
    min_length = min(len(feature1), len(feature2))
    feature1, feature2 = feature1[:min_length], feature2[:min_length]
    
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [23]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.decomposition import PCA

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]  # 무음 제거

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = intensity.values.T[0]

    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):
        formants_at_time = [formant.get_value_at_time(i, t) for i in range(1, 4)]
        formants_at_time = [f for f in formants_at_time if f is not None]
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.array(formant_values)
    
    return pitch_values, intensity_values, formant_values

# 3. 차원 축소(PCA) 적용
def apply_pca(features, n_components=1):
    pca = PCA(n_components=n_components)
    if len(features.shape) == 1:
        features = features.reshape(-1, 1)
    reduced_features = pca.fit_transform(features)
    return reduced_features.flatten()

# 4. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # DTW를 위한 벡터 길이 맞추기
    min_length = min(len(feature1), len(feature2))
    feature1, feature2 = feature1[:min_length], feature2[:min_length]
    
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 5. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출 및 PCA 적용
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # PCA를 적용하여 1차원으로 축소
    tts_pitch = apply_pca(tts_pitch)
    tts_intensity = apply_pca(tts_intensity)
    tts_formant = apply_pca(tts_formant)
    user_pitch = apply_pca(user_pitch)
    user_intensity = apply_pca(user_intensity)
    user_formant = apply_pca(user_formant)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


c:\Users\user\anaconda3\envs\4th_math\lib\site-packages\sklearn\decomposition\_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [24]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.decomposition import PCA

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]  # 무음 제거
    pitch_values = np.nan_to_num(np.ravel(pitch_values), nan=0.0)  # NaN을 0으로 대체하고 1차원 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.nan_to_num(np.ravel(intensity.values), nan=0.0)  # NaN을 0으로 대체
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.nan_to_num(np.ravel(np.array(formant_values)), nan=0.0)  # NaN을 0으로 대체
    
    return pitch_values, intensity_values, formant_values

# 3. PCA 적용
def apply_pca(features, n_components=1):
    pca = PCA(n_components=n_components)
    # 2차원 배열로 변환 후 PCA 수행
    if len(features.shape) == 1:
        features = features.reshape(-1, 1)
    reduced_features = pca.fit_transform(features)
    return reduced_features.flatten()

# 4. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # 길이 조정
    min_length = min(len(feature1), len(feature2))
    feature1, feature2 = feature1[:min_length], feature2[:min_length]
    
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 5. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 예시 문장과 TTS 파일 생성
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    # TTS와 사용자 음성 파일의 특징 추출
    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # PCA 적용하여 차원 축소
    tts_pitch = apply_pca(tts_pitch)
    tts_intensity = apply_pca(tts_intensity)
    tts_formant = apply_pca(tts_formant)
    user_pitch = apply_pca(user_pitch)
    user_intensity = apply_pca(user_intensity)
    user_formant = apply_pca(user_formant)

    # DTW를 이용한 유사도 계산
    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    # TTS 파일 삭제
    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [25]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.decomposition import PCA

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]  # 무음 제거
    pitch_values = np.nan_to_num(np.ravel(pitch_values), nan=0.0)  # NaN을 0으로 대체하고 1차원 변환

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.nan_to_num(np.ravel(intensity.values), nan=0.0)  # NaN을 0으로 대체
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.nan_to_num(np.ravel(np.array(formant_values)), nan=0.0)  # NaN을 0으로 대체
    
    return pitch_values, intensity_values, formant_values

# 3. PCA 적용 후 평탄화
def apply_pca(features, n_components=1):
    pca = PCA(n_components=n_components)
    if len(features.shape) == 1:
        features = features.reshape(-1, 1)
    reduced_features = pca.fit_transform(features)
    return np.ravel(reduced_features)  # 1차원 배열로 평탄화

# 4. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    min_length = min(len(feature1), len(feature2))
    feature1, feature2 = feature1[:min_length], feature2[:min_length]
    
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 5. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    # PCA 적용 후 평탄화된 벡터로 변환
    tts_pitch = apply_pca(tts_pitch)
    tts_intensity = apply_pca(tts_intensity)
    tts_formant = apply_pca(tts_formant)
    user_pitch = apply_pca(user_pitch)
    user_intensity = apply_pca(user_intensity)
    user_formant = apply_pca(user_formant)

    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [26]:
import os
import numpy as np
import parselmouth
from gtts import gTTS
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# 1. TTS로 예시 문장 음성 파일 생성
def create_tts_audio(text, filename="example_tts.mp3"):
    tts = gTTS(text, lang="en")
    tts.save(filename)
    return filename

# 2. Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    
    # 피치 (Pitch) 추출
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]  # 무음 제거
    pitch_values = np.nan_to_num(pitch_values, nan=np.nanmean(pitch_values))  # NaN 보간

    # 강세 (Intensity) 추출
    intensity = snd.to_intensity()
    intensity_values = np.nan_to_num(intensity.values.flatten(), nan=np.nanmean(intensity.values))  # NaN 보간
    
    # 폼란트 (Formants) 추출
    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):  # 0.01초 간격으로 폼란트 추출
        formants_at_time = []
        for i in range(1, 4):  # 첫 세 개의 폼란트 사용
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.nan_to_num(formant_values, nan=np.nanmean(formant_values))  # NaN 보간
    
    return pitch_values, intensity_values, formant_values

# 3. DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    # 두 벡터의 길이 조정
    min_length = min(len(feature1), len(feature2))
    feature1, feature2 = feature1[:min_length], feature2[:min_length]
    
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 4. 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_audio_path)

    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    os.remove(tts_audio_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 음성 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [31]:
import os
import numpy as np
import parselmouth
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import pyttsx3
from pydub import AudioSegment

# MP3를 WAV로 변환하는 함수
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format="wav")
    return wav_path

# pyttsx3로 오프라인 TTS 음성 파일 생성
def create_tts_audio(text, filename="example_tts.wav"):
    engine = pyttsx3.init()
    engine.save_to_file(text, filename)
    engine.runAndWait()
    return filename

# Praat로 음성 파일에서 피치, 강세, 폼란트 등 특징 추출
def extract_features(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values = pitch_values[pitch_values > 0]
    pitch_values = np.nan_to_num(pitch_values, nan=np.nanmean(pitch_values))

    intensity = snd.to_intensity()
    intensity_values = intensity.values.flatten()
    intensity_values = np.nan_to_num(intensity_values, nan=np.nanmean(intensity_values))

    formant = snd.to_formant_burg()
    formant_values = []
    for t in np.arange(0, snd.duration, 0.01):
        formants_at_time = []
        for i in range(1, 4):
            formant_value = formant.get_value_at_time(i, t)
            if formant_value is not None:
                formants_at_time.append(formant_value)
        if formants_at_time:
            formant_values.append(np.mean(formants_at_time))
    formant_values = np.nan_to_num(formant_values, nan=np.nanmean(formant_values))

    return np.ravel(pitch_values), np.ravel(intensity_values), np.ravel(formant_values)

# DTW를 이용한 유사도 계산
def dtw_similarity(feature1, feature2):
    min_length = min(len(feature1), len(feature2))
    feature1, feature2 = feature1[:min_length], feature2[:min_length]
    distance, _ = fastdtw(feature1, feature2, dist=euclidean)
    return distance

# 메인 함수: TTS 파일 생성, 사용자 음성 파일과 비교
def main(user_audio_path):
    # 사용자 MP3를 WAV로 변환
    user_wav_path = "user_audio.wav"
    convert_mp3_to_wav(user_audio_path, user_wav_path)

    example_text = "This is a sample sentence for pronunciation evaluation."
    tts_audio_path = create_tts_audio(example_text)

    tts_pitch, tts_intensity, tts_formant = extract_features(tts_audio_path)
    user_pitch, user_intensity, user_formant = extract_features(user_wav_path)

    pitch_dtw_distance = dtw_similarity(tts_pitch, user_pitch)
    intensity_dtw_distance = dtw_similarity(tts_intensity, user_intensity)
    formant_dtw_distance = dtw_similarity(tts_formant, user_formant)

    print(f"Pitch DTW distance: {pitch_dtw_distance:.2f}")
    print(f"Intensity DTW distance: {intensity_dtw_distance:.2f}")
    print(f"Formant DTW distance: {formant_dtw_distance:.2f}")

    os.remove(tts_audio_path)
    os.remove(user_wav_path)

# 사용자 음성 파일 경로 설정 후 실행
user_audio_path = "user_audio.mp3"  # 사용자 MP3 파일 경로로 변경하세요
main(user_audio_path)


ValueError: Input vector should be 1-D.

In [32]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

# download and load all models
preload_models()

# generate audio from text
text_prompt = """
     Hi, There. This is yura speaking.
"""
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)

# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)


No GPU being used. Careful, inference might be very slow!


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\4th_math\lib\site-packages\bark\generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\4th_math\lib\site-packages\huggingface_hub-0.24.6-py3.8.egg\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\4th_math\lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to C:\Users\user/.cache\torch\hub\checkpoints\encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 103MB/s] 
100%|██████████| 25/25 [02:46<00:00,  6.66s/it]
